# Embeding

In [1]:
from gensim.models import word2vec

In [2]:
#这一个是负样本均衡过的
import hickle as hkl
x_val = hkl.load('./Gen_data/13_x_val.hkl')
y_val = hkl.load('./Gen_data/13_y_val.hkl')
x_train = hkl.load('./Gen_data/13_x_train.hkl')
y_train = hkl.load('./Gen_data/13_y_train.hkl')

In [3]:
import numpy as np

In [4]:
print np.shape(x_val)

(843, 4096, 2)


In [1]:
from sklearn.externals import joblib
import os
import random
import numpy
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.grid_search import GridSearchCV 

In [2]:
base_root='/home/yinqijin/tempdata/'

In [3]:
random.seed(666)

fin1  = open(base_root+'/training.csv','r')

pbin   = [ [],[],[],[],[] ]
nbin   = [ [],[],[],[],[] ]
bins   = []
neg    = []
pos    = []
for line in fin1:
	if line[0] == 'b':
		continue
	else:
		data  = line.split('"')
		if data[1] not in bins:
			bins.append(data[1])
		index = bins.index(data[1])
		data  = line.split(',')
		if data[7] == '1':
			pbin[index].append(line)
			pos.append(line)
		else:
			nbin[index].append(line)	


for i in range(len(bins)):
	nbinlen = len(nbin[i])
	temp    = random.sample(nbin[i],nbinlen/20)
	neg.extend(temp)

nnum   = len(neg)
pnum   = len(pos)
fnum   = len(neg[0].strip().split(',')) - 18

print nnum,pnum,fnum

1280 1291 60


In [32]:
fin2   = open(base_root+'enhancers.bed','r')
fin3   = open(base_root+'promoters.bed','r')
enhancers = []
promoters = []

arrays1 = numpy.zeros((nnum+pnum,100,2))
labels1 = numpy.zeros(nnum+pnum)

for line in fin2:
	data = line.strip().split()
	enhancers.append(data[3])

for line in fin3:
	data = line.strip().split()
	promoters.append(data[3])

enhancer_model = Doc2Vec.load(base_root+'enhancers_6_1_100.d2v')
promoter_model = Doc2Vec.load(base_root+'promoters_6_1_100.d2v')


for i in range(len(pos)):
        data = pos[i].strip().split(',')
        e_index = enhancers.index(data[5])
        p_index = promoters.index(data[10])
        prefix_enhancer = 'ENHANCERS_' + str(e_index)
        prefix_promoter = 'PROMOTERS_' + str(p_index)
        enhancer_vec = enhancer_model.docvecs[prefix_enhancer]
        promoter_vec = promoter_model.docvecs[prefix_promoter]
        enhancer_vec = enhancer_vec.reshape((100,1))
        promoter_vec = promoter_vec.reshape((100,1))
	arrays1[i] = numpy.append(enhancer_vec,promoter_vec,axis=1)
	labels1[i] = 1

for j in range(len(neg)):
        data = neg[j].strip().split(',')
        e_index = enhancers.index(data[5])
        p_index = promoters.index(data[10])
        prefix_enhancer = 'ENHANCERS_' + str(e_index)
        prefix_promoter = 'PROMOTERS_' + str(p_index)
        enhancer_vec = enhancer_model.docvecs[prefix_enhancer]
        promoter_vec = promoter_model.docvecs[prefix_promoter]
        enhancer_vec = enhancer_vec.reshape((100,1))
        promoter_vec = promoter_vec.reshape((100,1))  #.reshape((1,100))
	arrays1[i+j] = numpy.append(enhancer_vec,promoter_vec,axis=1)  #umn_stack
	labels1[i+j] = 0

In [33]:
print numpy.shape(arrays1)

(2571, 100, 2)


In [34]:
from numpy import random

In [35]:
index = range(len(labels1))
random.shuffle(index)
SPLIT_RATIO = 0.85
split_point =int(SPLIT_RATIO*len(labels1))


In [36]:
print split_point

2185


In [37]:
arrays1 = numpy.array(arrays1)
labels1 = numpy.array(labels1)

In [60]:
x_train , y_train=  arrays1[index[:split_point]] ,labels1[ index [:split_point]]
x_valid ,y_valid= arrays1[index[split_point:]] , labels1[index[split_point:]]

In [61]:
from keras.utils.np_utils import to_categorical

In [62]:
import pickle as pkl

In [63]:
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

In [43]:
savedata =open('./Gen_data/14_data.pkl','w')
pkl.dump(x_train,savedata)
pkl.dump(y_train,savedata)
pkl.dump(x_valid,savedata)
pkl.dump(y_valid,savedata)


In [44]:
loaddata = open('./Gen_data/14_data.pkl','r')
x_train_1=pkl.load(loaddata)
y_train_1=pkl.load(loaddata)
x_valid_1=pkl.load(loaddata)
x_valid_1=pkl.load(loaddata)

In [45]:
assert  all( y_train == y_train_1)

In [46]:
numpy.shape(x_train)

(2185, 100, 2)

# Network

In [47]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [48]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

In [49]:
kmer_input = Input(shape=(100,2), dtype='float32')

In [50]:
l_lstm =Bidirectional(LSTM(5,return_sequences=True))(kmer_input)

In [54]:
l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
l_flatten = Flatten()(l_lstm_drop)
l_att_drop = Dropout(0.5)(l_flatten)
l_dense = Dense(128)(l_att_drop)
l_dense_1 = Dense(32)(l_dense)
preds = Dense(2, activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_dense_1)
model  = Model (kmer_input,preds)

In [55]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 100, 2)        0                                            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 100, 10)       320         input_2[0][0]                    
____________________________________________________________________________________________________
dropout_7 (Dropout)              (None, 100, 10)       0           bidirectional_2[0][0]            
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 1000)          0           dropout_7[0][0]                  
____________________________________________________

In [ ]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=20, batch_size=100)

INFO (theano.gof.compilelock): Waiting for existing lock by process '73460' (I am process '125580')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/lock_dir


In [ ]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), nb_epoch=20, batch_size=100)